In [40]:
import pandas as pd
import numpy as np
from sodapy import Socrata
import os
import requests
import matplotlib.pyplot as plt
import pyproj
import datetime
import json
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
%matplotlib inline

In [83]:
acs_results_2016 = requests.get('https://api.census.gov/data/2016/acs/acs5?get=NAME,B01001_001E,B01001_011E,B01001_012E,B01001_035E,B01001_036E,B01001_002E,B02001_002E,B02001_003E,B03001_003E,B01002_001E,B25031_001E,B06011_001E,B06009_005E&for=place:*&key=91aa6a44edb890abeea862f2d9befb5d520a5cc4')


In [24]:
acs_results_2016.status_code

200

In [97]:
data = acs_results_2016.text
total_df = pd.read_json(data)

In [87]:
total_df.head()

,NAME,race_pop,population,m_25_29,m_30_34,f_25_29,f_30_34,total_male,total_white,total_black,total_hispanic,median_age,median_gross_rent,median_income,total_bachelor,state,place,year
1,"Abanda CDP, Alabama",151,151,0,24,18,0,71,138,13,0,32.9,-666666666,12908,0,01,00100,2016
2,"Abbeville city, Alabama",2627,2627,81,72,52,41,1359,1386,1211,39,50.4,514,17653,116,01,00124,2016
3,"Adamsville city, Alabama",4422,4422,168,95,63,91,2164,2147,2243,28,42.0,844,26402,301,01,00460,2016
4,"Addison town, Alabama",757,757,18,41,26,11,383,722,0,0,46.2,471,20765,64,01,00484,2016
5,"Akron town, Alabama",252,252,0,6,8,0,121,36,216,0,43.9,533,11875,5,01,00676,2016


In [98]:
total_df.columns = total_df.iloc[0]
total_df.drop([0], inplace = True) 
variables = {'B01001_001E':"population",'B01001_011E':"m_25_29", 'B01001_012E':'m_30_34', 
             'B01001_035E': "f_25_29", 'B01001_036E':"f_30_34",'B01001_002E':"total_male",'DP05_0066PE': "pct_hispanic",
             'B02001_002E': "total_white",'B02001_003E':"total_black",'B03001_003E': "total_hispanic", 'B01002_001E':"median_age",'B25031_001E':"median_gross_rent",
             'B06011_001E': "median_income",'B06009_005E':"total_bachelor"}
total_df.rename(columns = variables, inplace=True)
total_df['year'] = 2016


In [93]:
cols = ['population', 'f_25_29', 'm_25_29', 'f_30_34', 'm_30_34', 'total_male', 'total_black', 'total_white', 
       'total_hispanic', 'total_bachelor']
total_df[cols] = total_df[cols].apply(pd.to_numeric, errors='coerce')

total_df['pct_25_34'] = (total_df['f_25_29'] + total_df['m_25_29'] + total_df['f_30_34']+ total_df['f_30_34'])/total_df['population']
total_df['pct_male'] = total_df['total_male']/total_df['population']
total_df['pct_black'] = total_df['total_black']/total_df['population']
total_df['pct_white'] = total_df['total_white']/total_df['population']
total_df['pct_hispanic'] = total_df['total_hispanic']/total_df['population']
total_df['pct_bachelor'] = total_df['total_bachelor']/total_df['population']

In [94]:
cs = total_df['NAME'].str.split(",", expand = True)
total_df['city'] = cs[0]
total_df['state_name'] = cs[1]

In [78]:
st = pd.read_table('state_code.txt', sep = '|')

In [54]:
st.head()

,STATE,STUSAB,STATE_NAME,STATENS
0,1,AL,Alabama,1779775
1,2,AK,Alaska,1785533
2,4,AZ,Arizona,1779777
3,5,AR,Arkansas,68085
4,6,CA,California,1779778


In [99]:
st.rename(index = str, columns = {'STATE': 'state','STUSAB':'state_code', 'STATE_NAME': 'state_name'}, inplace = True)
st['state'] = st['state'].apply(lambda x: '{0:0>2}'.format(x))
st = st[['state', 'state_code']]

In [80]:
total_df = total_df.merge(st, how = 'left', on = 'state')

In [95]:
total_df[total_df['city'] == 'Riverside city']

,NAME,race_pop,population,m_25_29,m_30_34,f_25_29,f_30_34,total_male,total_white,total_black,total_hispanic,median_age,median_gross_rent,median_income,total_bachelor,state,place,year,pct_25_34,pct_male,pct_black,pct_white,pct_hispanic,pct_bachelor,city,state_name
3075,"Riverside city, California",318678,318678,12870,12352,12534,11066,158097,206638,19164,165670,31.4,1194,24420,25444.0,06,62000,2016,0.149166,0.496103,0.060136,0.648423,0.519866,0.079842,Riverside city,California
8878,"Riverside city, Iowa",1054,1054,61,14,64,13,474,1036,3,4,42.2,701,26960,77.0,19,67440,2016,0.143264,0.449715,0.002846,0.982922,0.003795,0.073055,Riverside city,Iowa
14457,"Riverside city, Missouri",3103,3103,87,210,85,183,1625,2360,284,166,36.7,642,19115,321.0,29,62156,2016,0.173381,0.523687,0.091524,0.760554,0.053497,0.103448,Riverside city,Missouri
20100,"Riverside city, Ohio",25036,25036,1237,776,975,749,12681,21466,1947,990,35.7,798,25958,1875.0,39,67468,2016,0.148187,0.506511,0.077768,0.857405,0.039543,0.074892,Riverside city,Ohio
25858,"Riverside city, Texas",495,495,0,11,9,6,219,363,121,10,53.1,834,27857,33.0,48,62408,2016,0.042424,0.442424,0.244444,0.733333,0.020202,0.066667,Riverside city,Texas


In [81]:
total_df.to_csv('city_data.csv', index= False)